In [148]:
import tensorflow as tf

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))



Num GPUs Available: 0


In [149]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [150]:

# Завантажуємо готовий датасет IMDB
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Параметри
max_words = 30000   # більше слів у словнику
max_len = 300       # довші тексти
embedding_dim = 256 # розмір embedding
# lstm_units = 256    # більше нейронів у LSTM


(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=max_words)



In [151]:
# Паддинг (вирівнювання довжини)
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)

In [152]:
# Модель
model = keras.Sequential([
    layers.Embedding(max_words, 128, input_length=max_len),
    layers.LSTM(64, dropout=0.3, recurrent_dropout=0.2),
    layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])


In [153]:
callbacks = [
    EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    ModelCheckpoint("best_imdb_model.h5", save_best_only=True)
]


In [154]:
# Навчання
history = model.fit(
    x_train, y_train,
    batch_size=32,
    epochs=5,
    validation_split=0.2,
    verbose=1,
    callbacks=callbacks
)


Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - accuracy: 0.6958 - loss: 0.5637

625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 217ms/step - accuracy: 0.7419 - loss: 0.5229 - val_accuracy: 0.7454 - val_loss: 0.5180
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.8439 - loss: 0.3731

625/625 ━━━━━━━━━━━━━━━━━━━━ 140s 224ms/step - accuracy: 0.8531 - loss: 0.3543 - val_accuracy: 0.8314 - val_loss: 0.3844
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - accuracy: 0.9020 - loss: 0.2611

625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 227ms/step - accuracy: 0.8947 - loss: 0.2699 - val_accuracy: 0.8436 - val_loss: 0.3802
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 144s 231ms/step - accuracy: 0.9286 - loss: 0.1918 - val_accuracy: 0.8376 - val_loss: 0.4070
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 145s 233ms/step - accuracy: 0.9111 - loss: 0.2303 - val_accuracy: 0.8390 - val_loss: 0.4236


In [155]:
# Тестування
score = model.evaluate(x_test, y_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

782/782 ━━━━━━━━━━━━━━━━━━━━ 46s 59ms/step - accuracy: 0.8336 - loss: 0.3974
Test loss: 0.3974251449108124
Test accuracy: 0.8335599899291992


In [156]:
# --- 5. Декодер слів (щоб перетворити числа в текст і назад) ---
word_index = keras.datasets.imdb.get_word_index()
index_word = {v+3: k for k, v in word_index.items()}
index_word[0] = "<PAD>"
index_word[1] = "<START>"
index_word[2] = "<UNK>"
index_word[3] = "<UNUSED>"


In [163]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def predict_review(text):
    # Токенізація своїх слів (IMDB словник англійський!)
    tokens = [word_index.get(word.lower(), 2) for word in text.split()]  # 2 = <UNK>
    tokens = pad_sequences([tokens], maxlen=max_len)
    prediction = model.predict(tokens)[0][0]
    if prediction > 0.5:
        print(f"Відгук позитивний 🙂 (ймовірність {prediction:.2f})")
    else:
        print(f"Відгук негативний 🙁 (ймовірність {1-prediction:.2f})")

In [164]:

# --- 7. Перевірка ---
predict_review("the movie was very interesting, exciting")
predict_review("the film is not interesting")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Відгук позитивний 🙂 (ймовірність 0.86)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Відгук позитивний 🙂 (ймовірність 0.74)
